# **Assignment-2 for CS60075: Natural Language Processing**

#### Instructor : Prof. Sudeshna Sarkar

#### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Prithwish Jana, Udit Dharmin Desai

#### Date of Announcement: 15th Sept, 2021
#### Deadline for Submission: 11.59pm on Wednesday, 22nd Sept, 2021 
#### Submit this .ipynb file, named as `<Your_Roll_Number>_Assn2_NLP_A21.ipynb`

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.  This dataset consists of 50k movie reviews (25k positive, 25k negative). You can download the dataset from https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews



Please submit with outputs. 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rajde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rajde\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rajde\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
#Load the IMDB dataset. You can load it using pandas as dataframe
#engine='python', error_bad_lines=False
df=pd.read_csv("..\Desktop\IMDB Dataset.csv")
df.loc[0][1]
df.shape

(50000, 2)

# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [3]:
stop_words = list(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

def remURL(text):
  return re.sub(r'http\S+', '', text)  

def remAlNu(text):
  regex = re.compile('[^a-zA-Z0-9 ]')
  return regex.sub('',text)

def remStWrds(text):
  tokens = text.split(" ")
  tokens_filtered= [word for word in tokens if not word in stop_words]
  return (" ").join(tokens_filtered)

def lemma(text):
  # tokens = text.split(" ")
  tokens=word_tokenize(text)
  afterlemmatize = [lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in tokens ]
  return (" ").join(afterlemmatize)

# df.head()
# print(df['review'][0])
# df['review'][0]=cleanhtml(df['review'][0])
# print(df['review'][0])
# print(df['review'][2132])
# print(remURL(df['review'][2132]))
# print(df['review'][2138])
# print(remAlNu(df['review'][2138]))
# print(remStWrds(df['review'][0]))
# print(lemma("playing"))
# print(lemma(df['review'][0]))

i=0
for index, row in df.iterrows():
   temp=row['review']
   temp=temp.lower()
   temp=cleanhtml(temp)
   temp=remURL(temp)
   temp=remAlNu(temp)
   temp=remStWrds(temp)
   temp=lemma(temp)
   row['review']=temp
#    i+=1
#    if(i%100==0):
#         print(i)

In [4]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
count=0
length=0
for index, row in df.iterrows():
   if(row['sentiment']=='positive'):
      count+=1
      length+=len(row['review'].split(" "))
print("Avg. length of sentence in Positive class: ",length/count)

count=0
length=0
for index, row in df.iterrows():
   if(row['sentiment']=='negative'):
      count+=1
      length+=len(row['review'].split(" "))
print("Avg. length of sentence in Negative class: ",length/count)

Avg. length of sentence in Positive class:  121.14076
Avg. length of sentence in Negative class:  118.56368


# Naive Bayes classifier

In [5]:
# get reviews column from df
reviews = df['review']

# get labels column from df
labels = df['sentiment']

In [6]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
print(encoded_labels[:20])
# print(enc.classes_)


[1 1 1 0 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 0]


In [7]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.
#,random_state=0, stratify=labels

train_sentences, test_sentences, train_labels, test_labels= train_test_split(reviews, encoded_labels, test_size=0.20, random_state=0,  stratify=labels)

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [8]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words
'''
max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(Sentence_list)
'''
vec=CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences)
features=vec.get_feature_names()

frequency=X.toarray()
pos_freq=[0 for i in range(3000)]
neg_freq=[0 for i in range(3000)]

len(frequency)
for i in range(40000):
    if(train_labels[i]==1):
        for j in range(3000):
            pos_freq[j]+=frequency[i][j]
    else:
        for j in range(3000):
            neg_freq[j]+=frequency[i][j]
print(frequency.sum(axis=0))     

total_pos_features=0
total_neg_features=0

for i in range(3000):
    total_pos_features+=pos_freq[i]
    total_neg_features+=neg_freq[i]

[3398  699  325 ...  459 1496  217]


In [9]:
# Use laplace smoothing for words in test set not present in vocab of train set
def prob(elem):
    i=0
    for i in range(3000):
        if(elem==features[i]):
            break
        else:
            i+=1
    
    if i<3000:
        return ((pos_freq[i]+1)/(total_pos_features+3000))/((neg_freq[i]+1)/(total_neg_features+3000))
    else:
        return (total_neg_features+3000)/(total_pos_features+3000)

In [10]:
# Build the model. Don't use the model from sklearn
def modelNaive(text):
    lis=text.split(" ")
    pr=1
    for x in lis:
        pr=pr*prob(x)
        #print(pr)
    if pr>=1.0:
        return 1
    else:
        return 0

In [11]:
# Test the model on test set and report Accuracy
#train_sentences, test_sentences, train_labels, test_labels= train_test_split(reviews, encoded_labels, test_size=0.20, random_state=0,  stratify=labels)

match=0
for i in range(10000):
       #print(model(test_sentences.iloc[i]))
       #print(test_sentences.iloc[i])
       #if(i%100==0):
       #      print(i)
       if(modelNaive(test_sentences.iloc[i])==test_labels[i]):
            match+=1
       
        
print("Accuracy: ",match/100)            

Accuracy:  84.2


# *LSTM* based Classifier

Use the above train and test splits.

In [12]:
# Hyperparameters of the model
  # choose based on statistics
wordss=[]
for x in train_sentences:
    for y in x.split(" "):
        wordss.append(y)
vocab_size=len(set(wordss))
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 150 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [13]:
# tokenize sentences
#tokenizer = Tokenizer()
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [14]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 100)          17496600  
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 17,584,201
Trainable params: 17,584,201
Non-trainable params: 0
_________________________________________________________________


In [15]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 349s 301ms/step - loss: 0.4536 - accuracy: 0.7730 - val_loss: 0.2728 - val_accuracy: 0.8907
Epoch 2/5
1125/1125 [==============================] - 350s 311ms/step - loss: 0.1424 - accuracy: 0.9496 - val_loss: 0.3217 - val_accuracy: 0.8830
Epoch 3/5
1125/1125 [==============================] - 349s 310ms/step - loss: 0.0519 - accuracy: 0.9828 - val_loss: 0.4675 - val_accuracy: 0.8618
Epoch 4/5
1125/1125 [==============================] - 347s 309ms/step - loss: 0.0402 - accuracy: 0.9861 - val_loss: 0.4674 - val_accuracy: 0.8705
Epoch 5/5
1125/1125 [==============================] - 349s 310ms/step - loss: 0.0154 - accuracy: 0.9957 - val_loss: 0.5205 - val_accuracy: 0.8637


In [16]:
# Calculate accuracy on Test data



# Get probabilities
prediction = model.predict(test_padded)
print(prediction)

# Get labels based on probability 1 if p>= 0.5 else 0
for x in prediction:
    if x[0] >=0.5:
        x[0] = 1
    else:
        x[0] = 0
        
prediction = prediction.astype('int32') 
print(prediction)
# Accuracy : one can use classification_report from sklearn
print(classification_report(test_labels, prediction, labels=[0, 1]),sep='\n')

[[7.3085678e-01]
 [9.9996763e-01]
 [9.9874055e-01]
 ...
 [1.3092186e-05]
 [1.0930667e-04]
 [1.1824369e-03]]
[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]
              precision    recall  f1-score   support

           0       0.88      0.84      0.86      5000
           1       0.85      0.89      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



## Get predictions for random examples

In [17]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
preds=model.predict(padded)
print(preds)

# Get labels based on probability 1 if p>= 0.5 else 0
for x in preds:
    if x[0] >=0.5:
        x[0] = 1
    else:
        x[0] = 0
preds = preds.astype('int32') 
print(preds)

[[0.7759568 ]
 [0.04457322]
 [0.01447397]]
[[1]
 [0]
 [0]]
